# Setting up hadoop cluster


Setting up environment:

1. Oracle Box: Import Appliance, assign 1 CPU and 2GB memory. Save the imported VMs to a specific address on your drive by setting it up in base.
2. Don't forget to "reinitialize the MAC address for all network cards." (In Virtual Box version 6 and higher it is "Generate new MAC addresses for all network adapters.")
3. After importing them and renaming their group, change their network setting to host only.
4. These three VMs have two users: 
root with pass root
hadoop with pass hadoop
5. Check you host (in my case windows) IP by going to control panel>network and sharing center > Change adapter settign > Check ethernet 5 > properties > click on IPV4 and see your IP. In my case 192.168.56.1.
6. Back in master node (CentOS vm) open terminal from Applications? Favorites> Terminal
7. In terminal type nmtui to set the IP: 
disable IPV6;
make IPV4 manual and set it in the same IP range as your host machine;
set a host name: in this case node-master
8. in terminal type ifconfig to see if IP is set properly.
9. type exit to exit terminal.
10. In wired settings disable DNS and route.
11. From node-master ping host machine"
ping 192.168.56.1
12. From host machine ping node-master:
ping 192.168.56.10
13. Repeat steps 3 to 12 for all three nodes.
14. Open ssh terminal by mobaxterm to each of your machines. SSH port is 22.
15. To allow each node ping the other one with alias, we need to setup FQDN (Fully Qualified Domain Name). Consider that all nodes can see each other, but nodes do not see each others' names. To resolve this in windows: 

C:\Windows\System32\drivers\etc\hosts

Then edit hosts like below:

```
192.168.56.2 Node2
192.168.1.1 PC1 (This is my own laptop.)
192.168.56.10 node-master
192.168.56.11 node1
192.168.56.12 node2
```

This is not necessary for our windows machine (host), but for our nodes we need to do it.

On Linux nodes FQDN can set up on /etc/hosts
You may need to su to root user:
    ```
+ su
+ enter password for root
+ cd .. to get to root
+ cd etc
+ vi hosts

Hit insert button on key board to be able to add lines to hosts file. Add
```
192.168.56.10 node-master
192.168.56.11 node1
192.168.56.12 node2
```

Copy this file to all the other nodes or repeat step 15 for all of them.

To edit with vi, hit insert to be able to edit the file. To quit without saving use Esc, then type :q! To save and quit hit Esc, then type :wq (Write and Quit).

16. You can ssh from one node to another. For example on ndoe-master you can:
+ ssh node1
Enter password and you are connected to node1.

With user hadoop, you will be in /home/hadoop.

17. To connect passwordless to your servers:
I want user hadoop from node-master be able to connect to ndoe1 and node2 with ssh without password.
To do so (passwordless ssh):
+ open a session to node-master with the user you want to connect (in this case, hadoop).
+ On node-master with user hadoop generate a key:
+ ssh-keygen -b 4096
4096 is number of bits of the key.
The generated key by default is at /home/hadoop/.ssh/id_rsa. You can cat it.
cat /home/hadoop/.ssh/id_rsa.pub

18- Copy the generated key (be carfeul, just public key) to all other nodes. This includes master as well.
+ ssh-copy-id -i /home/hadoop/.ssh/id_rsa.pub hadoop@node-master

For the first time it will ask for password, but afterwards you can ssh passwordless.

19- Repeat step 18 for node1 and node2:
+ ssh-copy-id -i /home/hadoop/.ssh/id_rsa.pub hadoop@node1

If you have n nodes, you have to do it for all of them.

20- For nodes to be able to communicate with each other they need to see each others' IP and ports. At this stage we need to open firewalls.

Add the ports to firewall:

In hadoop 3 we need to open 9870, 9871, 9866, 9867 etc for different services of hadoop. We need to do it on each server for each port.

+ sudo firewall-cmd —zone=public —add-port=9000/tcp —permanent
+ sudo firewall-cmd reload

The ports we need to open for the beginning are as follows:

```
namenode HTTP Port: 9870
Namenode HTTPS Port: 9871
Datanode HTTP Port: 9864
Datanode HTTPS Port: 9865
SecondaryNN HTTP Port: 9868
SecoondaryNN HTTPS Port: 9869
```